# Newton line search on exponential fit (no JIT)

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path('..') / 'src'))
from kl_decomposition import rectangle_rule, fit_exp_sum, newton_with_line_search
from kl_decomposition.kernel_fit import _prepare_numpy_funcs
import numpy as np

In [ ]:
b_ls = np.array([0.5, 1])
N_max = 5

x, w = rectangle_rule(0.0, 2.0, 1000)
func = lambda t: np.exp(-t)

for N in range(3, N_max + 1):
    print(f'Fitting with N={N}')
    if len(b_ls) == 1:
        init_means = np.hstack([b_ls, b_ls[-1]**2])
    else:
        init_means = np.hstack([b_ls, b_ls[-1]**2 / b_ls[-2]])
    init_sigmas = np.ones((N,))
    a_ls, b_ls, info = fit_exp_sum(N, x, w, func, method='de_ls', compiled=False, 
                                   max_gen=100, pop_size=20, de_mean=init_means, de_sigma=init_sigmas)
    print('initial a:', a_ls)
    print('initial b:', b_ls)

In [ ]:
target = func(x)
obj, grad, hess = _prepare_numpy_funcs(x, target, w, N, newton=True)
params0 = np.concatenate([a_ls, np.log(b_ls)])
params_opt, stats = newton_with_line_search(params0, obj, grad, hess, max_iter=100, compiled=False)
print('refined a:', np.exp(params_opt[:N]))
print('refined b:', np.exp(params_opt[N:]))
print('Newton iterations:', stats.iterations)

In [ ]:
import matplotlib.pyplot as plt

# Compute the hessian and gradient at params_opt
hess_opt = hess(params_opt)
grad_opt = grad(params_opt)
# Compute the Newton step direction
direction = -np.linalg.solve(hess_opt, grad_opt)

# Evaluate obj along the line: params0 + alpha * direction
alphas = np.linspace(-1, 2, 100)
obj_values = [obj(params0 + alpha * direction) for alpha in alphas]

plt.plot(alphas, obj_values)
plt.xlabel('alpha')
plt.ylabel('Objective function')
plt.title('Objective function along Newton step direction')
plt.grid(True)
plt.yscale('log')
plt.show()